# Getty Query Tutorial (Place of Birth)
In this notebook, we perform the similar hunger-for-knowledge procedure for place of birth in Wikidata and Getty Vocabulary. The main difference is that we generalize the query for single property to the property chain, this is, the path in kgtk.

In [2]:
import os
import json
import pandas as pd

from kgtk.functions import kgtk, kypher

## Step 0: Set up environment paths
Here we set needed Graph files and desired output files to environment paths.

In [14]:
# We will define environment variables to hold the full paths to the files as we will use them in the shell commands
kgtk_environment_variables = []

# Folder where database files store
data_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
os.environ['DATABASE'] = data_path
kgtk_environment_variables.append('DATABASE')

# Wikidata (all is much less than claims)
os.environ['WIKIDATA'] = data_path + "claims.tsv"
kgtk_environment_variables.append('WIKIDATA')

# Label file of Wikidata
os.environ['KGTK_LABEL_FILE'] = data_path + "labels.en.tsv"
kgtk_environment_variables.append('KGTK_LABEL_FILE')

# P31
os.environ['P31'] = data_path + "P31.tsv"
kgtk_environment_variables.append('P31')

# P279star
os.environ['P279STAR'] = data_path + "P279star.tsv"
kgtk_environment_variables.append('P279STAR')

# Folder of ULAN
ulan_path = data_path + "gvp/ULAN/"
os.environ['ULAN'] = ulan_path
kgtk_environment_variables.append('ULAN')

# Full file provided by Getty 
# Full file is much truncated in this process thus we will use concat files below instead.
# os.environ['ULAN_FULL'] = ulan_path + "full.tsv"
# kgtk_environment_variables.append('ULAN_FULL')

# File concatenated by used explicit files: ULAN Subjects, AgentMap, Biographies, TGN-Wikidata Align
# The suffix ID means we performed add-id operation to the graph to meet the requirement of paths command.
ulan_concat_path = ulan_path + "ulan.concat.id.tsv"
os.environ['ULAN_CONCAT_ID'] = ulan_concat_path
kgtk_environment_variables.append('ULAN_CONCAT_ID')

# ULAN-Wikidata alignment file, maps ULAN ID to Wikidata Qnode
ulan_wikialign_path = ulan_path + "wiki.align.tsv"
os.environ['ULAN_ALIGN'] = ulan_wikialign_path
kgtk_environment_variables.append('ULAN_ALIGN')

# Output
output_path = data_path + "gvp/pob_output/"
if not os.path.exists(output_path):
    os.mkdir(output_path)
os.environ['OUTPUT'] = output_path
kgtk_environment_variables.append('OUTPUT')

# Each file will be explained later in the procedure
output_names = {
    "wiki_pob": "wiki.pob.tsv",
    "wiki_unknown": "wiki.unknown.tsv",
    "pairs": "pairs.tsv",
    "paths": "paths.tsv",
    "paths_label": "paths.label.tsv",
    "getty_pob": "getty.pob.tsv",
    "getty_mapped_pob": "getty.mapped.tsv",
    "getty_agree": "getty.agree.tsv",
    "getty_disagree": "getty.disagree.tsv",
    "getty_disagree_mapped": "getty.disagree.mapped.tsv",
    "getty_known": "getty.known.tsv",
    "correct": "correct.tsv",
    "unknown": "unknown.tsv"
}

for key, value in output_names.items():
    variable = key.upper()
    os.environ[variable] = os.path.join(output_path, value)
    kgtk_environment_variables.append(variable)

for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

DATABASE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
WIKIDATA: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/claims.tsv"
KGTK_LABEL_FILE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/labels.en.tsv"
P31: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P31.tsv"
P279STAR: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P279star.tsv"
ULAN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/"
ULAN_CONCAT_ID: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/ulan.concat.id.tsv"
ULAN_ALIGN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/wiki.align.tsv"
OUTPUT: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/pob_output/"
WIKI_POB: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/pob_output/wiki.pob.tsv"
WIKI_UNKNOWN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/pob_output/wiki.unknown.tsv"
PAIRS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/pob_output/pairs.tsv"
PATHS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/pob_o

## Step 1: Query Wikidata
Query Wikidata for PoB for people that have an ULAN ID (92k people)

In [3]:
# count total poeple
# ULAN ID
kgtk("""
    query -i $ULAN_ALIGN 
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_627_c1.""node1"")"
0,92353


In [4]:
# Qnode
kgtk("""
    query -i $ULAN_ALIGN 
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c1.""node2"")"
0,92210


In [4]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $WIKIDATA
        --match 'w: (ulanid)-[]->(qnode), 
                 a: (qnode)-[:P19]->(pob)' 
        --return 'distinct ulanid as node1, "P19" as label, pob as node2'
        --limit 10
    / add-labels
    """)

CPU times: user 26.8 ms, sys: 24.2 ms, total: 51 ms
Wall time: 1min 19s


,node1,label,node2,label;label,node2;label
0,ulan:500023949,P19,Q90,'place of birth'@en,'Paris'@en
1,ulan:500123827,P19,Q1297,'place of birth'@en,'Chicago'@en
2,ulan:500223082,P19,Q842925,'place of birth'@en,'Edelény'@en
3,ulan:500108173,P19,Q45779,'place of birth'@en,'Pécs'@en
4,ulan:500072302,P19,Q390798,'place of birth'@en,'Szentendre'@en
5,ulan:500091345,P19,Q754776,'place of birth'@en,'Kremnica'@en
6,ulan:500059872,P19,Q1781,'place of birth'@en,'Budapest'@en
7,ulan:500020713,P19,Q1792,'place of birth'@en,'Gdańsk'@en
8,ulan:500160869,P19,Q34713,'place of birth'@en,'Salzburg'@en
9,ulan:500113074,P19,Q210205,'place of birth'@en,'Pest'@en


In [5]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $WIKIDATA
        --match 'w: (ulanid)-[]->(qnode), 
                 a: (qnode)-[:P19]->(pob)' 
        --return 'distinct ulanid as node1, "P19" as label, pob as node2'
        -o $WIKI_POB
    """)

CPU times: user 13.2 ms, sys: 10.5 ms, total: 23.8 ms
Wall time: 14.6 s


## Step 2: Record Known and Unknown

In [6]:
# count known ULAN IDs
kgtk("""
    query -i $WIKI_POB
        --match '(ulanid)-[]->(pob)' 
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_629_c1.""node1"")"
0,64785


In [3]:
# count known Qnodes
kgtk("""
    query -i $WIKI_POB $ULAN_ALIGN
        --match 'p: (ulanid)-[]->(nation), a: (ulanid)-[]->(qnode)' 
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c2.""node2"")"
0,64680


Filter unknow in ULAN-Wikidata alignment

In [7]:
%%time
kgtk("""
    ifnotexists -i $ULAN_ALIGN 
        --filter-on $WIKI_POB 
        --input-keys node1 
        --filter-keys node1 
        -o $WIKI_UNKNOWN
    """)

CPU times: user 4.03 ms, sys: 10.1 ms, total: 14.1 ms
Wall time: 2.47 s


In [8]:
# count unknown ULAN IDs
kgtk("""
    query -i $WIKI_UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_641_c1.""node1"")"
0,27568


In [4]:
# count unknown Qnodes
kgtk("""
    query -i $WIKI_UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_641_c1.""node2"")"
0,27530


## Step 3: Query Getty
Based on the known PoB for people, query Getty. Before we perform single property query (we can consider it as 1 hop paths query), this time, we generalize to paths between known pairs (Subjects, Objects). Similarily we will compute the distributions of found paths and pick the top 1 path (property chain). 

### 3-1: Build Pairs from Wikidata Results 
Pairs file is needed for performing kgtk paths operation as the query objects.

In [9]:
%%time
kgtk("""
    query -i $WIKI_POB
        --match '(ulanid)-[]->(pob)'
        --return 'distinct ulanid as source, pob as target'
        -o $PAIRS
    """)

CPU times: user 4.47 ms, sys: 9.83 ms, total: 14.3 ms
Wall time: 2.3 s


In [10]:
!head $PAIRS | column -ts $'\t'

source          target
ulan:500023949  Q90
ulan:500123827  Q1297
ulan:500223082  Q842925
ulan:500108173  Q45779
ulan:500072302  Q390798
ulan:500091345  Q754776
ulan:500059872  Q1781
ulan:500020713  Q1792
ulan:500160869  Q34713


### 3-2: Perform Paths Mapping
This step is the core step of this query and it is also the bottleneck of running this procedure.

In [ ]:
%%time
kgtk("""
    paths --max_hops 4
        --path-file $PAIRS
        --path-mode NONE 
        --path-source source
        --path-target target
        -i $ULAN_CONCAT_ID
        --statistics-only
        -o $PATHS
    """)

In [22]:
!head $PATHS

node1	label	node2	id
p0	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p0-0-0
p0	1	ulan:500023949-agent:gvp:biographyPreferred:ulan_bio:4000061264	p0-1-1
p0	2	ulan_bio:4000061264:schema:birthPlace:tgn:7008038-place	p0-2-2
p0	3	tgn:7008038-place:skos:exactMatch:Q90	p0-3-3
p1	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p1-0-4
p1	1	ulan:500023949-agent:gvp:biographyNonPreferred:ulan_bio:4000174196	p1-1-5
p1	2	ulan_bio:4000174196:schema:deathPlace:tgn:7008038-place	p1-2-6
p1	3	tgn:7008038-place:skos:exactMatch:Q90	p1-3-7
p2	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p2-0-8


### 3-3: Select the Top 1 Property Chain
Here we use Pandas to sort the top 1 property chain. 

In [11]:
# load paths.tsv file
paths = pd.read_csv(os.environ['PATHS'], sep='\t')
# process property: since paths command generate id as node2, we need to further extract property from it
paths.node2 = paths.node2.apply(lambda x: ':'.join(x.split(':')[2:4]))
# save processed property
paths.to_csv(os.environ['PATHS_LABEL'], sep='\t', index=False)
# join property chain for each found path
paths_concat = paths.groupby(paths['node1']).agg({'node2': lambda x: ' '.join(list(x))})
# sort the top 
paths_concat.value_counts()#.head(1)

node2                                                                 
foaf:focus gvp:biographyPreferred schema:birthPlace skos:exactMatch       13978
foaf:focus gvp:biographyNonPreferred schema:birthPlace skos:exactMatch     7932
foaf:focus gvp:biographyPreferred schema:deathPlace skos:exactMatch        4737
foaf:focus gvp:biographyNonPreferred schema:deathPlace skos:exactMatch     2033
dtype: int64

### 3-4: Query Getty Using the Property Chain
Query Getty for the values of the unknown people, with the found property chain: `(ulanid)-[foaf:focus]->()-[gvp:biographyPreferred]->()-[schema:birthPlace]->()-[skos:exactMatch]->(Qnode of PoB)`

In [12]:
%%time
kgtk("""
    query -i $WIKI_UNKNOWN $ULAN_CONCAT_ID
        --match 'unknown: (ulanid)-[]->(), 
                 ulan: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(pob)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "schema:birthPlace" AND
                 p3.label = "skos:exactMatch"'
        --return 'distinct ulanid as node1, "P19" as label, pob as node2'
        --limit 10
    / add-labels
    """)

CPU times: user 35.6 ms, sys: 32.3 ms, total: 67.9 ms
Wall time: 1min 27s


,node1,label,node2,label;label,node2;label
0,ulan:500000011,P19,Q84,'place of birth'@en,'London'@en
1,ulan:500000035,P19,Q36074,'place of birth'@en,'Queensland'@en
2,ulan:500000057,P19,Q90,'place of birth'@en,'Paris'@en
3,ulan:500000063,P19,Q12439,'place of birth'@en,'Detroit'@en
4,ulan:500000088,P19,Q1741,'place of birth'@en,'Vienna'@en
5,ulan:500000141,P19,Q21,'place of birth'@en,'England'@en
6,ulan:500000164,P19,Q18383,'place of birth'@en,'Providence'@en
7,ulan:500000168,P19,Q84,'place of birth'@en,'London'@en
8,ulan:500000320,P19,Q90,'place of birth'@en,'Paris'@en
9,ulan:500000387,P19,Q1761,'place of birth'@en,'Dublin'@en


In [13]:
%%time
kgtk("""
    query -i $WIKI_UNKNOWN $ULAN_CONCAT_ID
        --match 'unknown: (ulanid)-[]->(), 
                 ulan: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(pob)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "schema:birthPlace" AND
                 p3.label = "skos:exactMatch"'
        --return 'distinct ulanid as node1, "P19" as label, pob as node2'
        -o $GETTY_POB
    """)

CPU times: user 11.8 ms, sys: 5.23 ms, total: 17.1 ms
Wall time: 5.92 s


Count how many new results (count in ULAN ID) we found

In [14]:
kgtk("""
    query -i $GETTY_POB
        --match '(ulanid)-[]->(pob)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_639_c1.""node1"")"
0,2939


## Step 4: Record New Results

Map back to Wikidata

In [15]:
%%time
kgtk("""
    query -i $GETTY_POB $ULAN_ALIGN
        --match 'g: (ulanid)-[p]->(pob), 
                 w: (ulanid)-[]->(qnode)'
        --return 'distinct qnode as node1, p.label as label, pob as node2'
        -o $GETTY_MAPPED_POB
    """)

CPU times: user 5.78 ms, sys: 10.7 ms, total: 16.4 ms
Wall time: 2.86 s


Count how many new results (count in Qnode) we found

In [16]:
kgtk("""
    query -i $GETTY_MAPPED_POB
        --match '(qnode)-[]->(pob)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_640_c1.""node1"")"
0,2939


## Step 5: Validate with Wikidata Constraints

Count how many new results we found (Qnodes)

In [17]:
%%time
kgtk("""
    query -i $GETTY_MAPPED_POB $P31 $P279STAR \
        --match 'g: (node1)-[nodeProp]->(node2), P31: (node2)-[]->(nodex), P279star: (nodex)-[]->(par)' \
        --where 'par in ["Q618123", "Q3895768", "Q24334893", "Q2221906", "Q102496", "Q27096213"]' \
        --return 'distinct node1 as `node1`, nodeProp.label as `label`, node2 as `node2`' \
        -o $CORRECT
    """)

CPU times: user 5.76 ms, sys: 11.6 ms, total: 17.4 ms
Wall time: 3.24 s


In [19]:
%%time
kgtk("""
    ifnotexists -i $GETTY_MAPPED_POB \
        --filter-on $CORRECT \
        --input-keys node1 node2 \
        --filter-keys node1 node2
    """)

CPU times: user 11.8 ms, sys: 11.8 ms, total: 23.5 ms
Wall time: 2.92 s


,node1,label,node2


## Step 6: Record Unknown after Query
We record unknowns after query Getty.

In [20]:
kgtk("""
    ifnotexists -i $WIKI_UNKNOWN \
        --filter-on $GETTY_POB \
        --input-keys node1 \
        --filter-keys node1
        -o $UNKNOWN
    """)

In [21]:
# count in ULAN IDs
kgtk("""
    query -i $UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_628_c1.""node1"")"
0,24629


In [5]:
# count in Qnodes
kgtk("""
    query -i $UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_628_c1.""node2"")"
0,24597


# Step 7: Comparison of the Known Values
How often do the results from two knowledge graphs agree

In [3]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $ULAN_CONCAT_ID
        --match 'w: (ulanid)-[]->(), 
                 u: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(pob)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "schema:birthPlace" AND
                 p3.label = "skos:exactMatch"'
        --return 'distinct ulanid as node1, "P19" as label, pob as node2'
        -o $GETTY_AGREE
    """)

CPU times: user 3.52 ms, sys: 56.8 ms, total: 60.3 ms
Wall time: 10.7 s


Count how many agree in ULAN ID:

In [ ]:
kgtk("""
    query -i $GETTY_AGREE $WIKI_POB 
        --match 'g: (ulanid)-[]->(pob), w: (ulanid)-[]->(pob)'
        --return 'count(distinct ulanid)'
    """)

Count how many agree in Qnode:

In [4]:
kgtk("""
    query -i $GETTY_AGREE $WIKI_POB $ULAN_ALIGN
        --match 'g: (ulanid)-[]->(pob), p: (ulanid)-[]->(pob), a: (ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c3.""node2"")"
0,13610


Count value per node:

In [5]:
nodes = kgtk("""
    query -i $GETTY_AGREE $ULAN_ALIGN 
        --match 'g: (ulanid)-[]->(), w: (ulanid)-[]->(qnode)'
        --return 'count(distinct qnode) as node'
    """)

In [6]:
import subprocess

values = subprocess.check_output("wc -l < $GETTY_AGREE", shell=True)
values = values.decode("utf-8").strip()
values = int(values) - 1

In [7]:
values / nodes.iloc[0]['node']

1.000315308213779

Filter out disagreement statements:

In [9]:
kgtk("""
    ifexists -i $GETTY_AGREE \
        --filter-on $WIKI_POB \
        --input-keys node1 \
        --filter-keys node1
        -o $GETTY_KNOWN
    """)

In [10]:
kgtk("""
    ifnotexists -i $GETTY_KNOWN \
        --filter-on $WIKI_POB \
        --input-keys node1 node2 \
        --filter-keys node1 node2
        -o $GETTY_DISAGREE
    """)

In [15]:
kgtk("""
    query -i $GETTY_DISAGREE $ULAN_ALIGN 
        --match 'g: (ulannode)-[p]->(v), w: (ulannode)-[]->(qnode)' 
        --return 'qnode as node1, p.label as label, v as node2'
        -o $GETTY_DISAGREE_MAPPED
    """)